# 9.1 インストール
```bash
$ pip install tensorflow
$ pip install tensorflow-gpu
```

# 9.2 最初のグラフの作成とセッション内での実行
TensorFlow では以下のような流れで計算を実行する．
- 1. 計算グラフの構築
- 2. セッションを開き計算を行う  
  
計算グラフを構築するだけでは変数の初期化さえ行われていない．セッションを開きオペレーションを CPU や GPU に載せて変数値を保持し計算を行う．

In [1]:
import tensorflow as tf

# 計算グラフを作る
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

# TensorFlow セッションを開き実際に計算する
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

# セッションを閉じる
sess.close()

42


with ブロックでセッションを開いて実行すれば，最後にセッションを閉じる必要はない．  
また，sess.run(f) と f.eval() は同じで計算の実行を担っている．

In [2]:
# 別の方法で実行する
# 最後に close しなくて良い
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    
print(result)

42


tf.global_variables_initializer() を使えば変数を一括して初期化できる．

In [3]:
# global_variables_initializer() のインスタンスを作成
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()  # すべての変数を初期化
    result = f.eval()
    
print(result)

42


tf.InteractiveSession() で自動的にデフォルトセッションを呼び出せるが，最後はマニュアルで close する必要がある．  

In [4]:
# IntaractiveSession() を使えば with ブロックは不要
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

# ただし close しないといけない
sess.close()

42


# 9.3 グラフの管理
作成したノードは自動的にデフォルトグラフに追加される．  
複数の計算グラフを管理する場合には，tf.Graph() を呼び出して新しいグラフを作り，with ブロックで一時的にデフォルトグラフに設定する．   
tf.reset_default_graph() を呼び出せばデフォルトグラフをリセットできる．

In [5]:
x1 = tf.Variable(1)
print(x1.graph is tf.get_default_graph())

# 複数のグラフを作る場合には、以下のように新しい Graph を作れば良い
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
True
False


# 9.4 ノードの値のライフサイクル
すべてのノードの値はグラフを実行するたびに破棄される．したがって，下記のようなコードでは w と x はそれぞれ2回ずつ計算されることになる．

In [6]:
tf.reset_default_graph()

# TensorFlow は自動的にノードの依存関係を判断してくれる
w = tf.constant(3)
x = w + 2
y = x + 5
z = y * 3

with tf.Session() as sess:
    # w と x は自動的に評価され、最後に y が計算される
    print(y.eval())
    # w と x の評価結果は再利用されず、再度計算される
    print(z.eval())

10
30


上のような計算方法では w と x を2回評価するため非効率．  
下記のように変更することで1回の実行で y と z が同時に評価される．

In [7]:
# w と x を2回評価するのは非効率なので、1回の実行で y と z が同時に評価されるようにする
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
30


# 9.5 TensorFlow による線形回帰
カリフォルニアの住宅価格データセットを使って線形回帰問題を解いてみる．  
線形回帰問題は以下の正規方程式を解くことでパラメータを計算できる．  
$$
    \hat{\theta} = ({\bf X}^T \cdot {\bf X})^{-1} \cdot {\bf X}^T \cdot {\bf y}
$$

In [8]:
# カリフォルニアデータセットを使って TensorFlow で線形回帰を試す
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

# データの読み込み
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data] # バイアス入力特徴量（x_0 = 1）を追加

In [9]:
tf.reset_default_graph()

# 計算グラフの構築
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

# 線形回帰で求めるパラメータ θ を求める
with tf.Session() as sess:
    theta_value = theta.eval()
    
print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [10]:
# scikit-learn でも同じように線形回帰してみる
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [11]:
# NumPy でも計算してみる
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


# 9.6 勾配降下法の実装
バッチ勾配降下法（第４章）を用いながら TensorFlow の勾配降下法について見ていく

In [12]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# データの読み込み
housing = fetch_california_housing()
m, n = housing.data.shape

# 前処理で scikit-learn を使って標準化を行う
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# エポック数と学習率の設定
n_epochs = 1000
learning_rate = 0.01

## 9.6.1 マニュアルの勾配降下法
- tf.random_uniform() 関数は与えられた形状の乱数を格納したテンソルを生成する
- tf.assign() 関数によって $ \theta_i = \theta_{i-1} - \eta\nabla_{\theta}MSE(\theta) $ の式でパラメータを更新する
- 100エポックごとに平均二乗誤差の値をチェック

In [13]:
tf.reset_default_graph()

# 最小二乗法による線形回帰問題の計算グラフの構築
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

# 計算の実行
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 3.7794578
Epoch 100 MSE = 0.6773614
Epoch 200 MSE = 0.5937237
Epoch 300 MSE = 0.5748632
Epoch 400 MSE = 0.5621264
Epoch 500 MSE = 0.5527556
Epoch 600 MSE = 0.5458045
Epoch 700 MSE = 0.54062843
Epoch 800 MSE = 0.5367596
Epoch 900 MSE = 0.53385633
Best theta:
[[ 2.0685523e+00]
 [ 7.2577614e-01]
 [ 1.2394900e-01]
 [-2.3193812e-02]
 [ 8.5017316e-02]
 [-1.2647666e-03]
 [-3.7551686e-02]
 [-9.5950717e-01]
 [-9.1584450e-01]]


## 9.6.2 自動微分を使った方法
数式微分（symbolic differentiation）なら偏微分方程式は見つけられるが，面倒で間違いやすい上，得られるコードは必ずしも効率的ではない   
　⇛ TensorFlow の自動微分機能を使えば自動的に効率的な方法で勾配計算を行える  
<br/>
**微分計算の方法**
- 数値微分    
　実装は簡単だが正確性に欠ける
- 数式微分   
　正確な計算が可能だが全く別の計算グラフを構築しなければならない
- フォワードモード自動微分    
　二重数（dual number）を使う
- リバースモード自動微分    
　出力次元+1回のトレースで正確な計算が可能  

<br/>
TensorFlow ではリバースモード自動微分が使われている．詳細は付録D（p.509）を参照のこと

In [14]:
tf.reset_default_graph()

# 計算グラフ上で gradients = ・・・ を tf.gradients() を使って書き換えるだけ
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

# 計算の実行
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

# マニュアル実装したときとほぼ同じ結果が得られる
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 11.239544
Epoch 100 MSE = 0.75668186
Epoch 200 MSE = 0.57880616
Epoch 300 MSE = 0.56168354
Epoch 400 MSE = 0.55123794
Epoch 500 MSE = 0.54373837
Epoch 600 MSE = 0.5383291
Epoch 700 MSE = 0.53442705
Epoch 800 MSE = 0.53161216
Epoch 900 MSE = 0.5295814
Best theta:
[[ 2.0685523 ]
 [ 0.7900566 ]
 [ 0.13563702]
 [-0.14532326]
 [ 0.18630771]
 [ 0.00215307]
 [-0.03991107]
 [-0.8139463 ]
 [-0.77779794]]


## 9.6.3 オプティマイザを使うと
TensorFlow には様々なオプティマイザが実装済み

In [15]:
tf.reset_default_graph()

# 計算グラフ上で gradients = ・・・ と training_op = ・・・ を書き換え
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# この先を書き換える
# 単純な勾配降下法
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# Momentum（こっちのほうが収束が早い）
optimizer = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9)
training_op = optimizer.minimize(mse)

# 計算の実行
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

# マニュアル実装したときとほぼ同じ結果が得られる
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 10.165731
Epoch 100 MSE = 0.53168565
Epoch 200 MSE = 0.52467155
Epoch 300 MSE = 0.52435696
Epoch 400 MSE = 0.5243256
Epoch 500 MSE = 0.5243216
Epoch 600 MSE = 0.524321
Epoch 700 MSE = 0.524321
Epoch 800 MSE = 0.524321
Epoch 900 MSE = 0.524321
Best theta:
[[ 2.0685577 ]
 [ 0.8296133 ]
 [ 0.11875058]
 [-0.2655158 ]
 [ 0.305687  ]
 [-0.00450332]
 [-0.03932605]
 [-0.89989907]
 [-0.8705537 ]]


# 9.7 訓練アルゴリズムへのデータの供給
イテレーションごとにミニバッチを入れ替える必要のあるミニバッチ勾配降下法では，プレースホルダーノードを使って実装すると良い．  
プレースホルダーノードでは計算は行われず，計算の実行時に指定したデータを出力する．  
一般に TensorFlow に訓練データを渡すときに用いられる．

In [16]:
# placeholder の定義
A = tf.placeholder(tf.float32, shape=(None, 3))  # None にしておけば可変長入力が取れる
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [19]:
tf.reset_default_graph()

# 9.6 で扱った勾配降下法をミニバッチ勾配降下法に書き換える
# 計算グラフは最初の2行を書き換え
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
# 以下は変更なし
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# ハイパーパラメータの設定
n_epochs = 1000
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

# ランダムにバッチサイズの大きさのデータを切り抜いてくる
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)  # 0~m-1 の乱数を batch_size 分だけ発生
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch
    

# 計算の実行
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            MSE = sess.run(mse, feed_dict={X:scaled_housing_data_plus_bias, y:housing.target.reshape(-1, 1)})
            print("Epoch", epoch, "MSE =", MSE)
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    
    best_theta = theta.eval()
    
print(best_theta)

Epoch 0 MSE = 10.625005
Epoch 100 MSE = 0.5355432
Epoch 200 MSE = 0.54215705
Epoch 300 MSE = 0.55939466
Epoch 400 MSE = 0.52536535
Epoch 500 MSE = 0.5284939
Epoch 600 MSE = 0.5277436
Epoch 700 MSE = 0.531569
Epoch 800 MSE = 0.52720755
Epoch 900 MSE = 0.5846888
[[ 2.0714476 ]
 [ 0.8462012 ]
 [ 0.11558536]
 [-0.26835835]
 [ 0.32982785]
 [ 0.00608358]
 [ 0.07052912]
 [-0.8798858 ]
 [-0.86342514]]


# 9.8 モデルの保存と復元
訓練済みのモデルを他のプログラムで使えるように，また訓練中にクラッシュしても大丈夫なように，定期的にチェックポイントを保存しておくのが望ましい．  
TensorFlow では Saver ノードを用いて保存と復元を行うことができる．  
- モデルの保存：tf.train.Saver.save()
- モデルの復元：tf.train.Saver.restore()

In [23]:
# 9.7 のモデルを100エポックごとに保存できるようにする
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "./tmp/my_model.ckpt")  # 保存
            MSE = sess.run(mse, feed_dict={X:scaled_housing_data_plus_bias, y:housing.target.reshape(-1, 1)})
            print("Epoch", epoch, "MSE =", MSE)
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")
    
print(best_theta)

Epoch 0 MSE = 12.440496
Epoch 100 MSE = 0.5355432
Epoch 200 MSE = 0.54215705
Epoch 300 MSE = 0.55939466
Epoch 400 MSE = 0.52536535
Epoch 500 MSE = 0.5284939
Epoch 600 MSE = 0.5277436
Epoch 700 MSE = 0.531569
Epoch 800 MSE = 0.52720755
Epoch 900 MSE = 0.5846888
[[ 2.0714476 ]
 [ 0.8462012 ]
 [ 0.11558536]
 [-0.26835835]
 [ 0.32982785]
 [ 0.00608358]
 [ 0.07052912]
 [-0.8798858 ]
 [-0.86342514]]


In [24]:
with tf.Session() as sess:
    # 上で保存したモデルを復元
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "./tmp/my_model.ckpt")  # 保存
            MSE = sess.run(mse, feed_dict={X:scaled_housing_data_plus_bias, y:housing.target.reshape(-1, 1)})
            print("Epoch", epoch, "MSE =", MSE)
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")
    
print(best_theta)

INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt
Epoch 0 MSE = 0.537573
Epoch 100 MSE = 0.5355432
Epoch 200 MSE = 0.54215705
Epoch 300 MSE = 0.55939466
Epoch 400 MSE = 0.52536535
Epoch 500 MSE = 0.5284939
Epoch 600 MSE = 0.5277436
Epoch 700 MSE = 0.531569
Epoch 800 MSE = 0.52720755
Epoch 900 MSE = 0.5846888
[[ 2.0714476 ]
 [ 0.8462012 ]
 [ 0.11558536]
 [-0.26835835]
 [ 0.32982785]
 [ 0.00608358]
 [ 0.07052912]
 [-0.8798858 ]
 [-0.86342514]]


Saver はどの変数をどの名前で保存・復元するか細かく設定できる．  
また，save() メソッドでは .meta ファイルにグラフ構造を保存する．このグラフ構造は tf.train.import_meta_graph() を使って読み込むことができる．

In [ ]:
# theta 変数だけを weights の名前で保存する
saver = tf.train.Savar({"weights": theta})

# グラフ構造の読み込み
saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")

# 9.9 TensorBoard を使ったグラフと訓練曲線の可視化
TensorBoard を使うことで統計情報やグラフをビジュアライゼーションを行うことで，グラフ中の誤り，ボトルネックなどを発見しやすくなる．そのためには，グラフの定義と訓練の統計情報を実行ごとに違ったログディレクトリに書き込むようにする必要がある．

In [28]:
tf.reset_default_graph()

# ログディレクトリの設定
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "log"
logdir = "./{}/run_{}/".format(root_logdir, now)

# 計算グラフの構築
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# コードを追加
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# 計算の実行
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            # 定期的に mse_sammary の評価を行い，イベントファイルにサマリを書き込む
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    file_writer.close()
    
print(best_theta)

[[ 2.0714476 ]
 [ 0.8462012 ]
 [ 0.11558536]
 [-0.26835835]
 [ 0.32982785]
 [ 0.00608358]
 [ 0.07052912]
 [-0.8798858 ]
 [-0.86342514]]


TesorBoard ウェブサーバーを起動し，ポート6006をリッスンすることで TensorBoard の統計情報やグラフの確認ができるようになる．

```bash
    $ tensorboard --logdir log/
```
<\br>
また，jupyter notebook 上でグラフの表示等を行う場合は，tensorboard.py を用意して呼び出すことで表示できるようになる．

```python
    import tensorflow as tf
    import tensorboard as tb
    
    [...]
    
    with tf.Session() as sess:
        [...]
        tf.summary.FileWriter('./log/', sess.graph)

    tb.show_graph(tf.get_default_graph().as_graph_def())
```

# 9.10 名前スコープ
名前スコープ（name scope）関連するノードをグループにまとめることができる．
例えば以下のように書くことで loss という名前スコープを定義できる．

In [29]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    
print(error.op.name)
print(mse.op.name)

loss/sub
loss/mse


# 9.11 モジュール性
ReLU (Rectified Linear Units) を実装してみる．
$$
    h_{w,h}({\bf X}) = max({\bf X} \cdot {\bf w} + b, 0)
$$

In [44]:
tf.reset_default_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

# ReLU 関数の計算グラフ
w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")
z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z2, 0., name="relu2")

output = tf.add(relu1, relu2, name="output")

# 計算の実行
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    print(output.eval(feed_dict={X: [[4, 5, 6], [7, 8, 9]]}))

[[ 9.359695]
 [16.523823]]


上記の書き方では繰り返しが多く見通しが悪い．  
DRY（Don't Repeat Yourself）を実践するため，以下のような書き方をするのが望ましい．

In [46]:
tf.reset_default_graph()

import tensorboard as tb

# TensorBoard での表示用
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "log"
logdir = "./{}/run_{}/".format(root_logdir, now)

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

# 計算の実行とグラフの表示
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    print(output.eval(feed_dict={X: [[4, 5, 6], [7, 8, 9]]}))
    tf.summary.FileWriter(logdir, sess.graph)
    
tb.show_graph(tf.get_default_graph().as_graph_def())

[[14.819654]
 [24.971457]]


名前スコープを使うとグラフがよりスッキリする．

In [47]:
tf.reset_default_graph()

import tensorboard as tb

# TensorBoard での表示用
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "log"
logdir = "./{}/run_{}/".format(root_logdir, now)

def relu(X):
    # 名前スコープでまとめる
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

# 計算の実行とグラフの表示
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    print(output.eval(feed_dict={X: [[4, 5, 6], [7, 8, 9]]}))
    tf.summary.FileWriter(logdir, sess.graph)
    
tb.show_graph(tf.get_default_graph().as_graph_def())

[[0.]
 [0.]]


# 9.12 変数の共有
グラフの要素間で変数を共有する方法として最も簡単なのは，その変数を使う関数に引数として変数を渡す方法である．

In [51]:
tf.reset_default_graph()

# threshold が共有したい変数
def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")
    
threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

# 計算の実行とグラフの表示
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    print(output.eval(feed_dict={X: [[4, 5, 6], [7, 8, 9]]}))

[[ 6.818099]
 [10.804081]]


何度も使う変数をいちいち引数で渡すのは面倒なので，以下のような方法で関数に渡すことがある．
- モデル内のすべての変数を格納した辞書を作りすべての関数に渡す
- 個々のモージュールごとにクラスを作る
- 関数が初めて呼び出されたときに関数の属性として共有変数を設定する

In [52]:
tf.reset_default_graph()

# threshold が共有したい変数
def relu(X):
    with tf.name_scope("relu"):
        # 関数が初めて呼び出されたときに関数の属性として共有変数を設定する
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, relu.threshold, name="relu")

TensorFlow では，しばしば get_variable() 関数を用いる．  
共有変数がまだなければ作り，そうでなければ既存の変数を再利用する．  
variable_scope() の属性によって再利用か否かを判断する．

In [54]:
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))

以前の get_variable() 呼び出しでこの変数がすでに作られていれば例外を生成する．  
変数を再利用する場合には，変数スコープの reuse 属性を True にして明示的に示す必要がある．

In [55]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")
    
# 以下のような書き方でも同じ意味
with tf.variable_scope("relu"):
    scope.reuse_variable()
    threshold = tf.get_variable("threshold")

既存の”relu/threshold"変数を読み出し，存在しない場合や作成されていない場合は例外を生成する．  
下のコードでは最初に relu() 関数を定義し，relu/threshold 変数を作成している．

In [59]:
tf.reset_default_graph()

# TensorBoard での表示用
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "log"
logdir = "./{}/run_{}/".format(root_logdir, now)

# threshold が共有したい変数
def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable(name="threshold")  # 変数は再利用
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="relu")
    
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
# 変数の作成
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

# 計算の実行とグラフの表示
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    print(output.eval(feed_dict={X: [[4, 5, 6], [7, 8, 9]]}))
    tf.summary.FileWriter(logdir, sess.graph)
    
tb.show_graph(tf.get_default_graph().as_graph_def())

[[24.067772]
 [37.667355]]


以下のコードでは最初に　relu() を呼び出したときに変数を作成し，その後はその変数を再利用している．

In [61]:
tf.reset_default_graph()

# TensorBoard での表示用
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "log"
logdir = "./{}/run_{}/".format(root_logdir, now)

# threshold が共有したい変数
def relu(X):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, threshold, name="relu")
    
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1 or None)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

# 計算の実行とグラフの表示
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    print(output.eval(feed_dict={X: [[4, 5, 6], [7, 8, 9]]}))
    tf.summary.FileWriter(logdir, sess.graph)
    
tb.show_graph(tf.get_default_graph().as_graph_def())

[[ 6.031849]
 [10.174026]]


# 9.13 演習問題

### 1. 計算グラフの構築する利点と欠点  
利点：
- TensorFlow の自動微分が使える
- 複数スレッドでの並列処理を実行するのに必要なことを TensorFlow が処理できる
- 異なるデバイス間でも同じモデルを共有しやすくなる
- TensorBoard など便利なツールが使える  

欠点：  
- 習得が手間
- ステップ・バイ・ステップでのデバッグが難しい

### 2. a_val = a.eval(session=sess) と a_val = sess.run(a) は同じか？
全く同じである．

### 3. a_val, b_val = a.eval(session=sess), b.eval(session=sess) と a_val, b_val = sess.run([a, b]) は同じか？
前者はグラフを2回辿って計算を実行するが，後者はグラフを1回だけ辿って計算を実行するので異なる意味を持つ．

### 4. 同じセッションで2つのグラフは実行可能か？
不可能．2つのグラフをマージする必要がある．

### 5. 2つのスレッドで同じグラフを使ってセッションを開いたとき変数は共有されるか？
ローカル TensorFlow ではセッションごとに変数を管理するので，各セッションが変数のコピーを作成し保持し変数は共有されない．  
分散 TensorFlow ではクラスタの管理するコンテナに変数値が格納されるため，同じコンテナを使う場合には2つのセッション間で変数が共有される．

### 6. 変数はいつ初期化されいつ破棄されるか？
コードが初期化子を呼び出したときに初期化され，セッションの終了時に破棄される．
ただし，分散 TensorFlow では変数はクラスタのコンテナに格納されるため破棄されない．

### 7. プレースホルダーと変数の違いは何か？
- 変数：値を保持する
    - 何らかの値を返し，実行時には初期化する必要がある
    - 値を変更可能でありステートフルである
    - グラフを連続的に実行したとき同じ値を持つ
    - パラメータの保持のほか，一般的な目的に使える
    
- プレースホルダー：それ自体に値を保持しない
    - プレースホルダーに依存するオペレーションを実行するときにはプレースホルダーの値を与える必要がある
    - 訓練データやテストデータの受け渡しに用いられることが多い
    - 代入ノードへの値渡し，変数値の変更にも使える

### 8. プレースホルダーに依存したオペレーションをプレースホルダーを与えずに実行するとどうなるか？また，プレースホルダーに依存しない場合はどうか？
前者では例外が生じるが，後者では例外が生成されることはない．

### 9. グラフの実行時に渡せるのはプレースホルダーだけか？
オペレーションの出力値を渡すことも可能だが，実際に行うことは殆ど無い．

### 10. 実行フェーズで変数に任意の値を設定するにはどうすればよいか？
tf.assign() 関数を使って代入ノードを作り，実行時にプレースホルダーを使って変数に新しい値を設定することができる．

### 11. 10個の変数についてコスト関数の勾配を計算するときリバースモード自動微分，フォワードモード自動微分，数式微分では何回グラフをトラバースする必要があるか？
- リバースモード自動微分：2回
- フォワードモード自動微分：10回
- 数式微分：全く異なるグラフを構築するためトラバースしない

### 12. TensorFlow とミニバッチ勾配降下法を用いてロジスティック回帰を実装せよ
ロジスティック回帰をまだやっていないので後回し